- 观测序列（observations）：实际观测到的现象序列
- 隐含状态（states）：所有的可能的隐含状态
- 初始概率（start_probability）：每个隐含状态的初始概率
- 转移概率（transition_probability）：从一个隐含状态转移到另一个隐含状态的概率
- 发射概率（emission_probability）：某种隐含状态产生某种观测现象的概率

O(Tn^2),n为隐状态的数量，T为序列长度

In [2]:
states = ('Healthy', 'Fever')

observations = ('normal', 'cold', 'dizzy')

start_probability = {'Healthy': 0.6, 'Fever': 0.4}

transition_probability = {
    'Healthy': {'Healthy': 0.7,'Fever': 0.3},
    'Fever': {'Healthy': 0.4,'Fever': 0.6},
}

emission_probability = {
    'Healthy': {'normal': 0.5,'cold': 0.4,'dizzy': 0.1},
    'Fever': {'normal': 0.1,'cold': 0.3,'dizzy': 0.6},
}


def Viterbit(obs, states, start_p, trans_p, emi_p):
    """
    obs:观测序列; states:隐含状态; start_p:初始概率; 
    trans_p:转移概率; emi_p:发射概率
    
    return: max_path, list
    """
    # path[s] 以s结尾的path
    path = {s:[] for s in states} 
    # 当前观测值的 p
    curr_p = {}
    
    # 初始观测为obs[0]的，各隐状态对应的 p
    for s in states:
        curr_p[s] = start_p[s] * emi_p[s][obs[0]]
    
    # 前向计算寻找每一组相邻时刻的 p 最大求解
    for i in range(1, len(obs)):
        # 上一个时刻的 p
        last_p = curr_p
        # 当前时刻的 p
        curr_p = {}
        
        # 前向计算该时刻各种可能隐状态的 p
        for curr_state in states:
            # 遍历上一个时刻的各个可能隐状态，对应的最大的 p，
            # 并记录此刻最大的 p 对应的上一个时刻的的隐状态
            max_p, last_sta = max(
                (
                    (
                    last_p[last_state] * trans_p[last_state][curr_state] * emi_p[curr_state][obs[i]], 
                    last_state
                    ) for last_state in states
                )
            )
            curr_p[curr_state] = max_p
            path[curr_state].append(last_sta)

    # 根据最后一个时刻的max_p，选择path
    # 对应路线，由path字典全纪录(Dynamic Programming)
    max_p = 0
    max_path = None
    for s in states:
        path[s].append(s)
        if curr_p[s] > max_p:
            max_path = path[s]
            max_p = curr_p[s]
    return max_path


if __name__ == '__main__':
    obs = ['normal', 'cold', 'dizzy']
    print(Viterbit(obs, states, start_probability, transition_probability, emission_probability))

['Healthy', 'Healthy', 'Fever']


```java
/**
 * 维特比算法
 * @author hankcs
 */
public class Viterbi
{
    /**
     * 求解HMM模型
     * @param obs 观测序列
     * @param states 隐状态
     * @param start_p 初始概率（隐状态）
     * @param trans_p 转移概率（隐状态）
     * @param emit_p 发射概率 （隐状态表现为显状态的概率）
     * @return 最可能的序列
     */
    public static int[] compute(int[] obs, int[] states, double[] start_p, double[][] trans_p, double[][] emit_p)
    {
        double[][] V = new double[obs.length][states.length];
        int[][] path = new int[states.length][obs.length];

        for (int y : states)
        {
            V[0][y] = start_p[y] * emit_p[y][obs[0]];
            path[y][0] = y;
        }

        for (int t = 1; t < obs.length; ++t)
        {
            int[][] newpath = new int[states.length][obs.length];

            for (int y : states)
            {
                double prob = -1;
                int state;
                for (int y0 : states)
                {
                    double nprob = V[t - 1][y0] * trans_p[y0][y] * emit_p[y][obs[t]];
                    if (nprob > prob)
                    {
                        prob = nprob;
                        state = y0;
                        // 记录最大概率
                        V[t][y] = prob;
                        // 记录路径
                        System.arraycopy(path[state], 0, newpath[y], 0, t);
                        newpath[y][t] = y;
                    }
                }
            }

            path = newpath;
        }

        double prob = -1;
        int state = 0;
        for (int y : states)
        {
            if (V[obs.length - 1][y] > prob)
            {
                prob = V[obs.length - 1][y];
                state = y;
            }
        }

        return path[state];
    }
}
```